# Homework 3 - MongoDB

In [36]:
import json
import os
import pymongo
import pandas as pd
from datetime import date, datetime, timedelta

## Lettura dei dati

In [2]:
dataset_dir = './dataset/'
dati_covid_path = os.path.join(dataset_dir, 'dpc-covid19-ita-regioni.json')
dati_vaccini_path = os.path.join(dataset_dir, 'somministrazioni-vaccini-latest.json')

dati_covid = []
with open(dati_covid_path, 'r') as j:
    dati_covid = json.load(j)

dati_vaccini = []
with open(dati_vaccini_path, 'r') as j:
    dati_vaccini = json.load(j)
dati_vaccini = dati_vaccini['data']

## Creazione del db
Si crea il database e due collection contenenti rispettivamente dati relativi al Covid-19 e alle vaccinazioni.

In [3]:
client = pymongo.MongoClient('localhost', 27017)

db = client['homework3']

collection_covid = db.dati_covid
collection_vaccini = db.dati_vaccini

print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'local']
Available collections:  []


Si inseriscono i dati nelle rispettive collection (solo se il database è stato appena creato).

In [4]:
# collection_covid.insert_many(dati_covid)
# collection_vaccini.insert_many(dati_vaccini)

In [5]:
print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'homework3', 'local']
Available collections:  ['dati_vaccini', 'dati_covid']


## Preprocessing
Si convertono i campi relativi alle date dal formato `string` al formato `date`.

In [8]:
collection_covid.aggregate([
    { "$addFields": {
        "data": {
            "$toDate": "$data"
            }
        } 
    },
    { "$unset": ["casi_da_screening", ]},
    { "$out" : "dati_covid" }
])

In [9]:
collection_vaccini.aggregate([
    { "$addFields": {
        "data_somministrazione": {
            "$toDate": "$data_somministrazione"
            }
        } 
    },
    { "$out" : "dati_vaccini" }
])

## Query

### Attuali positivi e terapie intensive per regione

In [86]:
days_ago = 4
date = datetime.today() - timedelta(days=days_ago)

start = date.replace(hour=0, minute=0, second=0)
end = date.replace(hour=23, minute=59, second=59)

filter = {
    'data': {
        '$gt': start,
        '$lt': end
    }
}

project = {
    'data': 1, 
    'denominazione_regione': 1, 
    'totale_positivi': 1,
    'terapia_intensiva': 1,
    '_id': 0
}

sort = list({
    'totale_positivi': -1
}.items())

result = collection_covid.find(
    filter=filter,
    projection=project,
    sort=sort
)

result_list = list(result)
# len(result_list)

In [85]:
df = pd.DataFrame(result_list)
df

,data,denominazione_regione,terapia_intensiva,totale_positivi
0,2021-05-14 17:00:00,Campania,102,80651
1,2021-05-14 17:00:00,Puglia,139,39241
2,2021-05-14 17:00:00,Lombardia,411,38689
3,2021-05-14 17:00:00,Lazio,231,34741
4,2021-05-14 17:00:00,Emilia-Romagna,160,26892
5,2021-05-14 17:00:00,Sicilia,120,18009
6,2021-05-14 17:00:00,Veneto,106,16735
7,2021-05-14 17:00:00,Toscana,182,14974
8,2021-05-14 17:00:00,Sardegna,39,14550
9,2021-05-14 17:00:00,Calabria,29,12345


### Nuovi positivi nell'ultima settimana per regione

In [132]:
weeks_ago = 1
date = datetime.today() - timedelta(weeks=1)

result = collection_covid.aggregate([
    { "$match": { "data": { "$gte": date } } },

    { "$group": {
        "_id": "$denominazione_regione",
        "nuovi_casi": { "$sum": "$nuovi_positivi" }
    }},
    
    { "$sort" : { "nuovi_casi": -1 }
    }
])

result_list = list(result)
# len(result_list)

In [133]:
df = pd.DataFrame(result_list)
df

,_id,nuovi_casi
0,Lombardia,4542
1,Campania,4464
2,Sicilia,2677
3,Lazio,2628
4,Piemonte,2604
5,Puglia,2444
6,Emilia-Romagna,2134
7,Toscana,2042
8,Veneto,1831
9,Calabria,1097
